# OpenStudio Refrigeration System Modeling and JSON Generator 
# Step-by-Step Guide

<span style="font-size:18px"> This notebook provides a structured workflow to generate and export OpenStudio-compatible JSON files for supermarket refrigeration systems. 

**Key modules:**
- Compressor and performance curve generation
- Condenser and fan curve logic
- Rack assignment based on thermal loads
- Case and Walk-in object creation
- Full refrigeration system assembly and export

</span>

## Practical Applications

This framework can be applied to multiple real-world use cases:

- **🔧 Practical Refrigeration Design**  
  Engineers and energy consultants can use the generated components to design and evaluate supermarket refrigeration systems under different templates (old, new, advanced).

- **🏗️ OpenStudio Energy Modeling**  
  The JSON objects generated here are fully compatible with OpenStudio's v0.2.1 schema, allowing integration into broader building energy simulation models. This is especially useful for load estimation, retrofit analysis, and performance benchmarking.

By automating object creation and tying it to real data and configurable templates, this tool serves as a bridge between design-level thinking and simulation-level precision.

### Mode Selection: How to Start
The modeling framework provides **two modes of operation** for flexibility and ease of use:

- **Automated Mode (Automated)**  
  This default mode automatically sets up a predefined SuperMarket configuration. It includes rack assignments, case/walk-in units, and refrigerant templates tailored for typical SuperMarket systems.  
  ✅ Ideal for quick scenario evaluations or template-based simulations.

- 🎛️ **User-Defined Mode (Manual Input)**  
  This mode allows the user to interactively choose specific refrigeration cases and walk-ins, define custom system configurations, and select desired templates.  
  ✅ Best for detailed, user-controlled modeling and custom design cases.

> These modes make the tool versatile for both **practical refrigeration system design** and **OpenStudio simulation workflows**, helping users make informed retrofit or maintenance decisions.

## Step 1: Imports & Setup
Import required modules and set your database path.

In [ ]:
import sys
sys.path.append(".")
from refrigeration.mode_selection import (
    automated_mode,
    user_mode,
    get_valid_template,
    select_test_mode
)
from refrigeration.rack_assignment import (
    assign_racks_to_cases_and_walkins,
    display_rack_capacity
)
from refrigeration.compressor import (
    summarize_compressor_assignment,
    prepare_and_store_compressor_objects,
    load_and_print_compressor_curves
)
from refrigeration.condenser import prepare_and_store_condenser_objects
from refrigeration.case_walkin_objects import (
    generate_case_objects_from_data,
    generate_walkin_objects_from_data,
    prepare_and_store_case_and_walkin_objects
)
from refrigeration.system_objects import (
    prepare_and_store_system_and_casewalkin_lists,
    generate_system_and_casewalkin_lists
)
from refrigeration.json_io import (
    export_existing_compressors_to_json,
    export_existing_condensers_to_json,
    export_cases_and_walkins_to_json,
    export_system_and_casewalkin_lists_to_json
)
from refrigeration.full_export import export_full_refrigeration_system_to_json
from refrigeration.utils import get_building_name, clean_name, generate_available_units_markdown

# set DB path 
db_path = "database/openstudio_refrigeration_system.db"

## Step 2: Select Mode (User / Automated)
Run either user_mode() or automated_mode() to proceed.

In this step, you’ll choose which refrigeration and freezer units to include and define the system template (Old, New, or Advanced).

- In **User mode**, you can manually select the units you want.

- In **Automated mode**, a default setup for the SuperMarket will be loaded automatically.

The selected template determines assumptions for performance curves and temperature settings, which will be used in later simulations and analysis.

<h2>🧊 Available Refrigeration Units</h2>
<h3>🔹 Available Cases</h3>

<div style="display: flex; justify-content: space-between; gap: 4%;">

  <div style="width: 48%;">
    <h4>Old/New System Templates</h4>
    <ul>
      <li>LT Coffin - Frozen Food</li>
      <li>LT Coffin - Ice Cream</li>
      <li>LT Reach-in - Frozen Food</li>
      <li>LT Reach-in - Ice Cream</li>
      <li>MT Island - Deli Produce</li>
      <li>MT Reach-in - Dairy Deli Beverage Case</li>
      <li>MT Service - Meat Deli Bakery</li>
      <li>MT Vertical Open - All</li>
    </ul>
  </div>

  <div style="width: 48%;">
    <h4>Advanced System Template</h4>
    <ul>
      <li>LT Coffin - Frozen Food</li>
      <li>LT Coffin - Ice Cream</li>
      <li>LT Reach-in - Frozen Food</li>
      <li>LT Reach-in - Ice Cream</li>
      <li>MT Island - Deli Produce</li>
      <li>MT Reach-in - Meat</li>
      <li>MT Reach-in - Others</li>
      <li>MT Service - Meat</li>
      <li>MT Service - Others</li>
      <li>MT Vertical Open - Beverage</li>
      <li>MT Vertical Open - Meat</li>
      <li>MT Vertical Open - Others</li>
    </ul>
  </div>

</div>

<h3>🧊 Available Walk-ins</h3>
<h4>For Old, New, and Advanced System Templates</h4>
<div style="display: flex; justify-content: space-between; gap: 4%;">

  <div style="width: 48%;">
    <ul>
      <li>LT Walk-in Freezer - 120SF</li>
      <li>LT Walk-in Freezer - 240SF</li>
      <li>LT Walk-in Freezer - 360SF</li>
      <li>LT Walk-in Freezer - 480SF</li>
      <li>LT Walk-in Freezer - 80SF</li>
      <li>MT Walk-in Cooler - 100SF with glass door</li>
      <li>MT Walk-in Cooler - 100SF with no glass door</li>
      <li>MT Walk-in Cooler - 120SF with glass door</li>
      <li>MT Walk-in Cooler - 120SF with no glass door</li>
      <li>MT Walk-in Cooler - 240SF with glass door</li>
      <li>MT Walk-in Cooler - 240SF with no glass door</li>
      <li>MT Walk-in Cooler - 360SF with glass door</li>
      <li>MT Walk-in Cooler - 360SF with no glass door</li>
    </ul>
  </div>

  <div style="width: 48%;">
    <ul>
      <li>MT Walk-in Cooler - 400SF with glass door</li>
      <li>MT Walk-in Cooler - 400SF with no glass door</li>
      <li>MT Walk-in Cooler - 480SF with glass door</li>
      <li>MT Walk-in Cooler - 480SF with no glass door</li>
      <li>MT Walk-in Cooler - 600SF with glass door</li>
      <li>MT Walk-in Cooler - 600SF with no glass door</li>
      <li>MT Walk-in Cooler - 64SF with glass door</li>
      <li>MT Walk-in Cooler - 64SF with no glass door</li>
      <li>MT Walk-in Cooler - 660SF with glass door</li>
      <li>MT Walk-in Cooler - 660SF with no glass door</li>
      <li>MT Walk-in Cooler - 80SF with glass door</li>
      <li>MT Walk-in Cooler - 80SF with no glass door</li>
    </ul>
  </div>

</div>

### 📌 User Selection Mode – Input Format
Each line should follow this format:

>$<$unit name$>$, $<$number of units$>$

**Examples**
- LT Coffin - Frozen Food, 2 
- MT Vertical Open - Beverage, 3
- LT Walk-in Freezer - 240SF, 1
- MT Walk-in Cooler - 360SF with glass door, 2

👉 Please refer to `example_user_mode.ipynb` and `example_automated_mode.ipynb` for two example scenarios.

In [ ]:
mode = select_test_mode()

if mode == "user":
    selected_case_units, selected_walkin_units, selected_template = user_mode()
elif mode == "automated":
    selected_case_units, selected_walkin_units, selected_template = automated_mode(db_path)

# Selected Case Units
print("\nSelected Case Units:")
for unit in selected_case_units:
    print(f"\"osm name\": \"{unit.osm_name}\", \"case_name\": \"{unit.case_name}\", \"number_of_units\": {unit.number_of_units}")

# Selected Walk-in Units
print("\nSelected Walk-in Units:")
for unit in selected_walkin_units:
    print(f"\"osm name\": \"{unit.osm_name}\", \"walkin_name\": \"{unit.walkin_name}\", \"number_of_units\": {unit.number_of_units}")

## Step 3: Rack Assignment
Determine the total Medium Temperature (MT) and Low Temperature (LT) load requirements for each rack.  
This is a critical preprocessing step used to size compressors appropriately based on system demand.

Using the `summarize_compressor_assignment()` function, it performs:

- ✅ Aggregation of cooling loads for each rack based on assigned cases and walk-ins
- ✅ Structuring the rack info as input to compressor generation logic

In [ ]:
# Assign refrigeration units to racks and retrieve updated data
mt_racks, lt_racks, case_data, walkin_data = assign_racks_to_cases_and_walkins(
    db_path, selected_case_units, selected_walkin_units
)

# display capacity distribution for assigned racks
display_rack_capacity(mt_racks, selected_case_units, rack_type="MT")
display_rack_capacity(lt_racks, selected_walkin_units, rack_type="LT")

# Step 4: Generate and Store Case and Walk-in objects
This step converts the **database-retrieved information** and **user-selected unit configurations** into OpenStudio-compatible `Refrigeration:Case` and `Refrigeration:WalkIn` JSON objects.

Using the `prepare_and_store_case_and_walkin_objects()` function, it performs:
- ✅ Mapping of each unit to its respective thermal zone
- ✅ Inclusion of case/walk-in properties like capacity, fan power, lighting, defrost schedule, etc.
- ✅ Output in OpenStudio object format for downstream JSON export

In [ ]:
result = prepare_and_store_case_and_walkin_objects(case_data, walkin_data, selected_case_units, selected_walkin_units)
case_objects = result["case_objects"]
walkin_objects = result["walkin_objects"]

## Preview and export Case and Walk-in object in JSON format
This step takes the `case_objects` and `walkin_objects` generated in Step 4 and combines them with the thermal zones into a single JSON structure.

Using the `export_cases_and_walkins_to_json()` function, it:

- ✅ Assigns the `MainSales` and `ActiveStorage` zones to refrigeration cases and walk-ins
- ✅ Adds all `Refrigeration:Case` and `Refrigeration:WalkIn` objects
- ✅ Outputs a clean, OpenStudio-compatible `.json` file for integration or preview.

In [ ]:
export_cases_and_walkins_to_json(case_objects, walkin_objects, output_path="All_Cases_Walkins.json")

# Step 5: Generate Compressor Objects
This step focuses on the creation of compressor objects and their associated performance curves, based on the assigned refrigeration racks and the selected system template.

Key operations include:
- Aggregating rack load information to determine compressor sizing and configuration.
- Retrieving performance curve data from the system database, aligned with the selected design template.
- Constructing `OS:Refrigeration:Compressor` objects and associated `OS:Curve:Bicubic` definitions suitable for OpenStudio simulation.
- Exporting all generated components into a structured JSON file for model preview, integration, or further analysis.

## Rack Assignment Summary
Determine the total Medium Temperature (MT) and Low Temperature (LT) load requirements for each rack.  
This step ensures compressors are correctly sized according to the system’s refrigeration demand.

Using the `summarize_compressor_assignment()` function, it:

✅ Aggregates cooling loads across all assigned cases and walk-ins  
✅ Separates data into MT and LT groups  
✅ Outputs rack-wise load information required for compressor object generation

In [ ]:
mt_info, lt_info = summarize_compressor_assignment(mt_racks, lt_racks, selected_template) 

## Compressor Curve Generation
This step retrieves the power and capacity performance curves for both MT and LT compressors based on the selected system template (`old`, `new`, or `advanced`).

Using the `load_and_print_compressor_curves()` function, it performs:

- ✅ Querying of compressor curve data from the database
- ✅ Extraction of `OS:Curve:Bicubic` JSON objects for both power and capacity
- ✅ Returns four curve objects ready for use in compressor creation

In [ ]:
mt_power_curve, mt_capacity_curve, lt_power_curve, lt_capacity_curve = \
    load_and_print_compressor_curves(db_path, selected_template)

## Generate and Store Compressor Objects
This step creates the actual compressor objects using the previously assigned rack loads and loaded performance curves, structured for OpenStudio compatibility.

Using the `prepare_and_store_compressor_objects()` function, it performs:

- ✅ Generation of `OS:Refrigeration:Compressor` objects for MT and LT racks
- ✅ Assignment of suction temperatures and compressor performance curves
- ✅ Returns objects as structured Python dictionaries ready for export

In [ ]:
result = prepare_and_store_compressor_objects(mt_info, lt_info, selected_template, db_path)
mt_compressors = result["mt_compressors"]
lt_compressors = result["lt_compressors"]
mt_power_curve = result["mt_power_curve"]
mt_capacity_curve = result["mt_capacity_curve"]
lt_power_curve = result["lt_power_curve"]
lt_capacity_curve = result["lt_capacity_curve"]

## Preview and export Compressor Objects in JSON format
This step takes the `mt_compressors`, `lt_compressors`, and their associated performance curves generated in Step 5 and merges them with thermal zones into a unified OpenStudio-compatible JSON structure.
Using the `export_existing_compressors_to_json()` function, it:

- ✅ Includes all `OS:Refrigeration:Compressor` objects
- ✅ Adds associated `OS:Curve:Bicubic` performance curves
- ✅ Assigns the `MainSales` and `ActiveStorage` thermal zones
- ✅ Outputs a clean, OpenStudio-compatible `.json` file for integration or preview.

In [ ]:
export_existing_compressors_to_json(
    mt_compressors=mt_compressors,
    lt_compressors=lt_compressors,
    mt_power_curve=mt_power_curve,
    mt_capacity_curve=mt_capacity_curve,
    lt_power_curve=lt_power_curve,
    lt_capacity_curve=lt_capacity_curve,
    output_path="All_Compressors.json"
)

# Step 6: Generate Condenser Objects
This step handles the creation of air-cooled condenser objects and their associated fan power curves based on the rack load and system type (MT or LT).
Key operations include:
- Calculate condenser capacity based on assigned rack loads.
- Generate `OS:Refrigeration:Condenser:AirCooled` objects for each rack.
- Create `OS:Curve:Linear` fan power curves linked to each condenser.
- Export everything to JSON format for preview or downstream use.

## Generate and store Condenser Objects
This step generates OpenStudio-compatible air-cooled condenser objects and their associated fan power curves.

Using the `prepare_and_store_condenser_objects()` function, it performs:

- ✅ Calculates condenser capacity and fan power based on MT/LT rack loads  
- ✅ Creates `OS:Refrigeration:Condenser:AirCooled` and corresponding `OS:Curve:Linear` objects  
- ✅ Outputs Python dictionaries for use in JSON export

In [ ]:
result = prepare_and_store_condenser_objects(mt_info, lt_info, selected_template)
mt_condensers = result["mt_condensers"]
lt_condensers = result["lt_condensers"]
mt_curves = result["mt_curves"]
lt_curves = result["lt_curves"]

## Preview and export Compressor objects in JSON format
This step generates condenser objects and their curves into a single OpenStudio-compatible JSON structure.

Using the `export_existing_condensers_to_json()` function, it:

- ✅ Includes all `OS:Refrigeration:Condenser:AirCooled` objects  
- ✅ Adds associated `OS:Curve:Linear` fan curves  
- ✅ Assigns the `MainSales` and `ActiveStorage` thermal zones  
- ✅ Outputs a clean, OpenStudio-compatible `.json` file for integration or preview.

In [ ]:
export_existing_condensers_to_json(
    mt_condensers=mt_condensers,
    lt_condensers=lt_condensers,
    mt_curves=mt_curves,
    lt_curves=lt_curves,
    output_path="All_Condensers.json"
)

# Step 7: Generate and Store Refrigeration Systems and Case/Walkin Lists
This step creates OpenStudio `OS:Refrigeration:System` objects and their associated `CaseAndWalkInList` objects by linking selected case/walk-in units to each compressor rack.

Using the `prepare_and_store_system_and_casewalkin_lists()` function, it performs:

- ✅ Maps each case and walk-in unit to its appropriate MT or LT rack  
- ✅ Assigns suction and condensing temperatures based on the system template  
- ✅ Generates `OS:Refrigeration:System` and `OS:Refrigeration:CaseAndWalkInList` objects  
- ✅ Returns Python objects for JSON export

In [ ]:
system_and_casewalkin_objects = prepare_and_store_system_and_casewalkin_lists(
    selected_case_units,
    selected_walkin_units,
    mt_racks,
    lt_racks,
    selected_template
)

## Preview and export Systems and Case/Walkin objects in JSON format
This step saves all generated refrigeration system objects and their case/walk-in lists into a clean OpenStudio-compatible JSON structure.

Using the `export_system_and_casewalkin_lists_to_json()` function, it:

- ✅ Includes all `OS:Refrigeration:System` and `OS:Refrigeration:CaseAndWalkInList` objects  
- ✅ Assigns all associated case/walk-in names per rack  
- ✅ Outputs a clean, OpenStudio-compatible `.json` file for integration or preview.

In [ ]:
export_system_and_casewalkin_lists_to_json(system_and_casewalkin_objects, "System_and_CaseWalkin_Lists.json")

# Step 8: Preivew and Export FULL Refrigeration system JSON Files
This final step consolidates all previously generated OpenStudio objects—including compressors, condensers, performance curves, refrigeration cases, walk-ins, and systems—into a single JSON file for full system integration or simulation.

Using the `export_full_refrigeration_system_to_json()` function, it:

- ✅ Combines all `OS:Refrigeration:Compressor`, `OS:Refrigeration:Condenser:AirCooled`, and related `OS:Curve` objects  
- ✅ Includes all `OS:Refrigeration:Case`, `OS:Refrigeration:WalkIn`, and their thermal zone assignments  
- ✅ Merges all `OS:Refrigeration:System` and `OS:Refrigeration:CaseAndWalkInList` components  
- ✅ Outputs a complete, OpenStudio-compatible `.json` file for simulation or design analysis


In [ ]:
export_full_refrigeration_system_to_json(
    mt_compressors=mt_compressors,
    lt_compressors=lt_compressors,
    mt_power_curve=mt_power_curve,
    mt_capacity_curve=mt_capacity_curve,
    lt_power_curve=lt_power_curve,
    lt_capacity_curve=lt_capacity_curve,
    mt_condensers=mt_condensers,
    lt_condensers=lt_condensers,
    mt_curves=mt_curves,
    lt_curves=lt_curves,
    case_objects=case_objects,
    walkin_objects=walkin_objects,
    system_and_casewalkin_objects=system_and_casewalkin_objects,
    output_path="Full_Refrigeration_System.json"
)

print("🏁 Refrigeration JSON export complete.")
print(f"Building: {get_building_name()}")